# Comparing m(> 2) Systems

## Two-way ANOVA

あるシステム$i$のトピック（クエリ）$j$に対する評価値（nDCG@10など）を$x_{ij}$とする．$x_{ij} = \mu + a_i + b_j + \epsilon_{ij}$ であると仮定する．ここで，$\mu$は定数，$a_i$はシステム効果（システム＝手法，と考えてもよい），$b_j$はトピック効果，$\epsilon_{ij}$は正規分布から得られるノイズ（無視して考えても良い）であるとする．つまり，評価値$x_{ij}$が上記の単純な式で得られると仮定してしまい，$a_1 = a_2 = \ldots = 0$ということであればシステムの効果はない，と結論づけよう，と考える．逆に，$a_1 = a_2 = \ldots = 0$ではない場合には，システム間に差がある，と結論づけよう，と考える．この，$a_1 = a_2 = \ldots = 0$を帰無仮説$H_0$とし，$H_0$が正しいと仮定した下で$x_{ij}$が得られることはどれほど珍しかったのかを求める．もし非常に珍しいようであれば$H_0$が正しいと仮定したことがそもそも間違っていたのではないかと考えこれを棄却する．これが$n$トピックに対する$m$システムの評価値に対する検定の概要である．

上記の仮説検定はTwo-way ANOVA (without replication)（繰り返しのない二元配置）によって行うことが可能である．
以下にTwo-way ANOVAをサンプルデータに適用する例を示す．

In [2]:
import pandas as pd

df = pd.read_csv("sample_data.csv")
df

,Topic,System,Score
0,Q1,A,0.2
1,Q1,B,0.4
2,Q1,C,0.3
3,Q2,A,0.3
4,Q2,B,0.4
5,Q2,C,0.5
6,Q3,A,0.4
7,Q3,B,0.4
8,Q3,C,0.5
9,Q4,A,0.2


上記のサンプルファイルは例えば，1つ目のクエリQ1に対して，Aシステム（A手法）のnDCG@10は0.2，BシステムのnDCG@10は0.4，CシステムのnDCG@10は0.3，であったということを意味している．これに対して，Two-way ANOVAを適用する．

なお，事前に `pip install statsmodels` を実行してパッケージをインストールする必要がある．

In [3]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
    
formula = 'Score ~ (System) + (Topic)'
model = ols(formula, df).fit()
aov_table = anova_lm(model, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
System,0.063333,2.0,10.230769,0.001826
Topic,0.112917,7.0,5.211538,0.004275
Residual,0.043333,14.0,NaN,NaN


上記のような結果が得られた場合，下記のように報告すれば良い：

> A two-way ANOVA revealed that the system effect is statistically significant ($F(2, 14) = 10.2$, $p<0.05$).

ここで，$F(\phi_A, \phi_E)$はANOVAで得られた表のうち，SystemのF値（System行のF列の値）であり，$\phi_A$はSystemの自由度（df列），$\phi_E$はResidualの自由度である．$p$値はSystem行のPR列の値をみて決定する．上記の0.05は有意水準であり，論文を通して同じ値を共通して使うべきである．情報検索においては0.05がよく見られるように思われる．

なお，上記の結果ではトピック効果も認められるがここでの主題ではないため言及していない．

## Tukey HSD

Two-way ANOVAによってわかることは単にH0でない，つまり，$a_1 = a_2 = \ldots = 0$でない，ことだけである．システム間に差があることは言えるが，どのシステム間に差があるかはわからない．単純な解決索はt検定などを単純に適用すればよいが，これには[多重比較問題](https://en.wikipedia.org/wiki/Multiple_comparisons_problem)という致命的な問題がある．そこで用いるべきは，Tukey HSDなど多重比較問題を補正する手法である．

Pythonでこれを行いたいところではあるが，2021年3月現在，1要因用のTukey HSDはあっても，2要因用のTukey HSDはないようである．下記のように，statsmodelsを利用した場合には1要因のTukey HSDとなってしまい帰無仮説が棄却されにくい．
そのため，**少なくともstatsmodelsを使うべきではなく，今のところPythonで2要因用のTukey HSDを行う術はないようである．**

```
>>> from statsmodels.stats.multicomp import pairwise_tukeyhsd
>>> tukeyhsd_result = pairwise_tukeyhsd(df.Score, df.System)
>>> print(tukeyhsd_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
===================================================
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     A      B    0.075 0.2146 -0.0337 0.1837  False
     A      C    0.125 0.0224  0.0163 0.2337   True
     B      C     0.05  0.491 -0.0587 0.1587  False
---------------------------------------------------
```

選択肢としては以下が考えられる：
- Rを利用する
- Randomized Tukey HSDを利用する: http://research.nii.ac.jp/ntcir/tools/discpower-ja.html

## RによるTukey HSD

Rで以下のように実行すれば良い：

```r
> data <- read.csv("sample_data.csv")
> TukeyHSD(aov(Score ~ System + Topic, data))

Tukey multiple comparisons of means
95% family-wise confidence level

Fit: aov(formula = Score ~ System + Topic, data = data)

$System
     diff          lwr      upr     p adj
B-A 0.075  0.002193978 0.147806 0.0431815
C-A 0.125  0.052193978 0.197806 0.0013729
C-B 0.050 -0.022806022 0.122806 0.2062942

...
```

上記のように，システムB-A間，システムB-A間，システムB-A間の調整済みp値が得られる．
有意水準α=0.05の下では，B-A間およびC-A間に有意差が見られる（p値が0.05より小さい）．

下記のように報告すると良い：

> A Tukey HSD test shows that the differences of A-B and A-C pairs are statistically significant ($p < 0.05$).

## Effect Size

しばしば見られる誤りとして，nDCG@10の差が0.1あるから良い，10%改善したから良い，p値が低いから良い，という主張がある．完全に間違いではないが正確ではない．例えば，他の手法間の差が高々0.001しかないときに，0.01の改善ができればその改善は大きな改善と言える．逆もまたしかりである．p値はトピック数（クエリ数）を増やせばいくらでも小さくすることができるため，p値の小ささを持って効果の大きさを主張することはできない．そこで用いられるべきなのは，**効果量**（Effect Size）である．元論文ではANOVAの効果量についても紹介されているが，ここでは主にシステム間の効果量だけを説明する．

Tukey HSDに基づく効果量は以下のように計算することができる：

$${\rm ES} = d / \sqrt{V_E}$$

ただし，$d$はシステム間の評価値の差の絶対値（Tukey HSDの表中のdiff列），$V_E = S_E / \phi_E$であり，$S_E$はANOVAで得られた表のResidual行のsum_sq列の値であり，$\phi_E$は$\phi_A$はResidualの自由度（df列）である．上記のANOVAの結果に基づくとシステムAとシステムCの効果量は以下のように計算できる．

$$V_E = S_E / \phi_E = 0.043333 / 14 = 0.00310 $$

$${\rm ES} = d / \sqrt{V_E} = 0.125 / \sqrt{0.00310} = 2.25$$


プログラムで求める場合には以下のようにすれば良い：

In [7]:
from math import sqrt

S_E = aov_table.sum_sq["Residual"]
V_E = S_E / aov_table.df["Residual"]

# 「各システムの評価値の平均」の差の絶対値．Tukey HSDのdiff列を利用
diffs = {
    "B-A": 0.075,
    "C-A": 0.125,
    "C-B": 0.050
}

for pair, diff in diffs.items():
    print(pair, diff / sqrt(V_E))

B-A 1.3480755514093758
C-A 2.246792585682293
C-B 0.8987170342729173


Tukey HSDの結果と合わせて報告する場合には下記のようにすれば良い：


> A Tukey HSD test shows that the differences of A-B and A-C pairs are statistically significant ($p < 0.05$).
> The effect size of these pairs is 1.35 for  2.24 for A-C

効果量には一応目安があり，上記の求めている効果量がCohen's dと等価だとすれば， https://en.wikipedia.org/wiki/Effect_size より，

|Effect size|ES|
|------|------|
|Very small|0.01|
|Small|0.20|
|Medium|0.50|
|Large|0.80|
|Very large|1.20|
|Huge|2.0|

である．